## RAG Pipeline with Gemini LLM, ChromaDB, and External PDF Knowledge

In [1]:
import google.generativeai as genai

In [2]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

warnings.filterwarnings("ignore")

In [18]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
### Load LLM Models

In [5]:
api_key = "AZzaByA78u1MehLi7Ymfy30c9wZALUgZEkGD_u0"

In [6]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=api_key,
                             temperature=0.2,top_p = 0.5 ,convert_system_message_to_human=True)

In [7]:
model

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.2, top_p=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000015DE7B23170>, default_metadata=(), convert_system_message_to_human=True)

### Custom Data

In [9]:
pdf_loader = PyPDFLoader("unit 6 all.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

Acquiring Editor:Todd Green
Editorial Project Manager:Lindsay Lawrence
Project Manager:Punithavathy Govindaradjane
Designer: Matthew Limbert
Morgan Kaufmann is an imprint of Elsevier
225 Wyman Street, Waltham, MA 02451, USA
Copyright r 2013 Elsevier Inc. All rights reserved
No part of this publication may be reproduced or transmitted in any form or by any means, electronic
or mechanical, including photocopying, recording, or any information storage and retrieval system, without
permission in writing from the publisher. Details on how to seek permission, further information about
the Publisher’s permissions policies and our arrangements with organizations such as the Copyright Clearance
Center and the Copyright Licensing Agency, can be found at our website:www.elsevier.com/permissions.
This book and the individual contributions contained in it are protected under copyright by the Publisher
(other than as may be noted herein).
Notices
Knowledge and best practice in this field are constan

### Embedding Mode and converting to vector knowledge
### Chroma DB uses a dense retriever. It is a vector database that stores embeddings and allows for efficient similarity search based on those embeddings.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [13]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

In [14]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

#### RetrievalQA.from_chain_type: This creates a retrieval-based question-answering chain. It combines a model with a retriever to answer questions by first retrieving relevant documents and then passing those documents to the model to generate an answer.

#### model: This is the language model (e.g., GPT, LLaMA, etc.) that will generate the answers based on the context provided by the retrieved documents.

#### retriever=vector_index: The retriever (vector_index) is used to search for and retrieve relevant documents based on the query. In this case, vector_index would be the vector store (like Chroma, FAISS, BM25, etc.) that stores the document embeddings and allows for similarity search to find documents most related to the query.

#### return_source_documents=True: This flag tells the system to not only return the answer to the question but also return the source documents that were used to generate the answer. This is useful for transparency and traceability, as you can see which documents the model relied on.

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True
)

In [19]:
question = "Describe parrallel and distributed computing in detail?"
result = qa_chain({"query": question})
# result["result"]
Markdown(result["result"])

**Parallel Computing**

* Involves dividing a computation into smaller tasks that can be executed concurrently on multiple processors.
* Focuses on maximizing performance within a single computer system.
* Types:
    * Single-instruction, single-data (SISD)
    * Single-instruction, multiple-data (SIMD)
    * Multiple-instruction, single-data (MISD)
    * Multiple-instruction, multiple-data (MIMD)

**Distributed Computing**

* Involves dividing a computation into tasks that can be executed on multiple independent computers connected by a network.
* Focuses on harnessing resources from multiple systems to solve complex problems.
* Architectural styles:
    * Client-server
    * Peer-to-peer
    * Cloud computing
    * Grid computing

**Key Differences**

| Feature | Parallel Computing | Distributed Computing |
|---|---|---|
| Execution | Within a single computer | Across multiple computers |
| Focus | Performance within a system | Resource utilization across systems |
| Communication | Shared memory or message passing | Message passing |
| Data | Shared or local | Distributed |
| Scalability | Limited by the number of processors in a system | Scalable to large numbers of computers |
| Fault tolerance | Limited | Higher due to redundancy |
| Cost | Lower | Higher due to network and hardware requirements |
| Applications | Scientific simulations, image processing | Data processing, web services, distributed databases |

**Advantages of Parallel and Distributed Computing**

* **Increased performance:** By distributing tasks across multiple processors or computers, computations can be completed faster.
* **Scalability:** Distributed systems can be easily scaled up or down to meet changing demands.
* **Fault tolerance:** Distributed systems are more fault-tolerant than centralized systems, as tasks can be reassigned to other computers in case of failure.
* **Resource utilization:** Distributed systems can efficiently utilize resources by sharing them among multiple tasks.
* **Cost-effectiveness:** Cloud computing and grid computing offer cost-effective ways to access computing resources on demand.

In [20]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Answer should be detailed and add example at the end if available 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [21]:
question = "Describe parllel and distributed computing in detail?"
result = qa_chain({"query": question})
Markdown(result["result"])

**Parallel Computing**

* Involves the simultaneous execution of multiple tasks on multiple processors within a single computer system.
* Aims to improve performance by dividing a problem into smaller subtasks that can be processed concurrently.
* Requires specialized hardware and software to coordinate the execution of tasks and manage shared resources.
* Examples: multi-core processors, graphics processing units (GPUs), and vector processors.

**Distributed Computing**

* Involves the execution of tasks across multiple independent computers connected by a network.
* Leverages the combined resources of multiple machines to solve complex problems or handle large workloads.
* Requires mechanisms for communication, coordination, and fault tolerance among the distributed components.
* Examples: cloud computing, grid computing, and distributed databases.

**Key Differences**

| Feature | Parallel Computing | Distributed Computing |
|---|---|---|
| Execution Environment | Single computer system | Multiple independent computers |
| Task Distribution | Within a single machine | Across multiple machines |
| Resource Sharing | Shared memory and resources | No shared memory, communication via network |
| Coordination | Centralized control | Decentralized control |
| Fault Tolerance | Limited to within the single machine | Requires mechanisms for handling failures across multiple machines |
| Scalability | Limited by the number of processors in the system | Can scale to very large numbers of machines |
| Applications | Scientific simulations, image processing, video encoding | Web services, data analytics, distributed storage |

**Advantages of Parallel and Distributed Computing**

* **Increased Performance:** By distributing tasks across multiple processors or machines, parallel and distributed computing can significantly improve the speed of computation.
* **Scalability:** Distributed computing allows for easy scaling of resources by adding or removing machines as needed.
* **Fault Tolerance:** Distributed systems can provide fault tolerance by replicating tasks across multiple machines, ensuring that failures in one machine do not affect the overall computation.
* **Cost-Effectiveness:** Cloud computing, a form of distributed computing, offers cost-effective access to computing resources on demand, eliminating the need for expensive hardware investments.